In [3]:
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

sinthetic_samples_by_class=16

In [55]:
def rank_classes(confusion_matrix, method='CCR', verbose=True):
    '''TODO: ccr
    TODO: uniform'''
    if method == 'CCR':
        ccrs = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=1)
        class_ranking = np.argsort(ccrs)
        return class_ranking, ccrs[class_ranking]
    
    elif method == 'max_confusion':
        off_diagonal_sum = np.sum(confusion_matrix, axis=1) - np.diag(confusion_matrix)
        class_ranking = np.argsort(off_diagonal_sum)[::-1]  
        return class_ranking, off_diagonal_sum[class_ranking]

    elif method == 'max_pairwise_confusion':
        m = confusion_matrix
        class_ranking = []
        print(m)
        while len(class_ranking)<m.shape[0]:
            if verbose:
                print(find_argmax_off_diagonal(m))
            (a,b) = find_argmax_off_diagonal(m)
            print(m[a,b])
            m[a,b] = 0
            m[b,a] = 0
            if a not in class_ranking:
                class_ranking.append(a)
            if b not in class_ranking:
                class_ranking.append(b)
            if verbose:
                print(a,b)
                print(m)
                print(class_ranking)
        return np.asarray(class_ranking), m
    else:
        raise ValueError("Unknown method: {}".format(method))
        
def find_argmax_off_diagonal(matrix):
    matrix_copy = np.array(matrix)
    np.fill_diagonal(matrix_copy, 0)
    # Finding the indices of the maximum off-diagonal value
    argmax_indices = np.unravel_index(np.argmax(matrix_copy, axis=None), matrix_copy.shape)
    return argmax_indices


In [56]:
X = np.random.rand(*(500, 6))
y_unique_values = [0, 1, 2, 3, 4]
y = np.random.choice(y_unique_values, size=(500,1))


# Create and fit the logistic regression model
clf = LogisticRegression(random_state=0, max_iter=100)
clf.fit(X, y)


pred = clf.predict(X)
cm = confusion_matrix(y, pred)
print('cm: ')
print(np.round(cm,2))

ranking, ccrs = rank_classes(cm, method='CCR')

print('Ranking: ', ranking)
print(type(ranking))


ranking, ccrs = rank_classes(cm, method='max_confusion')

print('Ranking: ', ranking)
print(type(ranking))

cm: 
[[20 14 10 18 32]
 [13 16 13 23 35]
 [15 11 14 22 34]
 [16 16 12 26 31]
 [10 21 13 18 47]]
Ranking:  [2 1 0 3 4]
<class 'numpy.ndarray'>
Ranking:  [1 2 3 0 4]
<class 'numpy.ndarray'>


/home/franciscoperez/anaconda3/envs/cnnpelsvae/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [57]:
ranking, ccrs = rank_classes(cm, method='max_pairwise_confusion')
print('Ranking: ', ranking)
print(type(ranking))

[[20 14 10 18 32]
 [13 16 13 23 35]
 [15 11 14 22 34]
 [16 16 12 26 31]
 [10 21 13 18 47]]
(1, 4)
35
1 4
[[20 14 10 18 32]
 [13 16 13 23  0]
 [15 11 14 22 34]
 [16 16 12 26 31]
 [10  0 13 18 47]]
[1, 4]
(2, 4)
34
2 4
[[20 14 10 18 32]
 [13 16 13 23  0]
 [15 11 14 22  0]
 [16 16 12 26 31]
 [10  0  0 18 47]]
[1, 4, 2]
(0, 4)
32
0 4
[[20 14 10 18  0]
 [13 16 13 23  0]
 [15 11 14 22  0]
 [16 16 12 26 31]
 [ 0  0  0 18 47]]
[1, 4, 2, 0]
(3, 4)
31
3 4
[[20 14 10 18  0]
 [13 16 13 23  0]
 [15 11 14 22  0]
 [16 16 12 26  0]
 [ 0  0  0  0 47]]
[1, 4, 2, 0, 3]
Ranking:  [1 4 2 0 3]
<class 'numpy.ndarray'>


In [58]:
#ADD TO RANK_CLASS

dict_priorization = {}
ranking_penalization = 2
for o in ranking:
    dict_priorization[o] =  int(sinthetic_samples_by_class*ranking_penalization)
    if ranking_penalization>0.5:
        ranking_penalization = ranking_penalization/2
print(dict_priorization)

{1: 32, 4: 16, 2: 8, 0: 8, 3: 8}
